In [1]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [15]:
from functools import reduce

import torch
import transformers
import pandas as pd

from tqdm.auto import tqdm

from new_semantic_parsing.utils import get_vocab_top_schema
from new_semantic_parsing.schema_tokenizer import TopSchemaTokenizer

In [3]:
data_path = '../data/top-dataset-semantic-parsing/'

train_data = pd.read_table(data_path + 'train.tsv', names=['text', 'tokens', 'schema'])

In [4]:
schema_vocab = reduce(set.union, map(get_vocab_top_schema, train_data.schema))

In [5]:
text_tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')
schema_tokenizer = TopSchemaTokenizer(schema_vocab, text_tokenizer)

In [16]:
text_ids = [text_tokenizer.encode(text) for text in tqdm(train_data.tokens)]


  7%|▋         | 2117/31279 [00:22<00:38, 750.65it/s]

In [21]:
schema_ids = []
pointer_masks = []

for i, schema in tqdm(enumerate(train_data.schema), total=len(train_data)):

    item = schema_tokenizer.encode_plus(schema, text_ids[i])
    schema_ids.append(item.ids)
    pointer_masks.append(item.pointer_mask)


In [8]:
# for i, schema in enumerate(train_data.schema):
#     try:
#         schema_tokenizer.encode_plus(schema, text_ids[i])
#     except AssertionError:
#         print(i)
#         print(schema)
#         print(text_ids[i])

In [18]:
from new_semantic_parsing.utils import PointerDataset

In [22]:
dataset = PointerDataset(text_ids, schema_ids, pointer_masks)

In [23]:
dataset.torchify()

In [26]:
torch.save({'train': dataset}, 'dataset.pth')

In [27]:
!rm dataset.pth

In [28]:
from pathlib import Path

[autoreload of new_semantic_parsing.schema_tokenizer failed: Traceback (most recent call last):
  File "/usr/local/google/home/vlialin/miniconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/google/home/vlialin/miniconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/local/google/home/vlialin/miniconda3/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/local/google/home/vlialin/miniconda3/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "

In [34]:
!mkdir tokenizer

In [35]:
text_tokenizer.save_pretrained('tokenizer')

('tokenizer/vocab.txt',
 'tokenizer/special_tokens_map.json',
 'tokenizer/added_tokens.json')

In [37]:
p.exists()

False